# Benchmarking Our Algorithm against others

In [2]:
import sys
sys.path.append('../../')
from time import time
import signal
from os import listdir
import numpy as np
import pickle as pkl


from panav.sequential import sequential_planning
from panav.SAMP.solvers import Tube_Planning, Simple_MILP_Planning

from benchmarks.ST_RRTStar_wrapper import sequential_ST_RRTStar 

%load_ext autoreload
%autoreload 2

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for ompl::msg::LogLevel already registered; second conversion method ignored.


In [3]:
def get_env_name(filename):
    fs = filename.split('.')
    return fs[0]


def run_experiments(env_name, 
                    env_info, 
                    N_range, 
                    algs, 
                    TIMEOUT = 100,
                    results = [], 
                    executed = None):
    
    '''
        Checkpointed results and timeouted can be passed in so we can resume operation.
    '''

    if executed is None:
        executed = {alg: {N :False for N in N_range} for alg in algs}

    for N in N_range: 
        # Construct the environment
        constructor = env_info['constructor']
        specs = env_info['specs']

        print('Building the environment...')
        env = constructor(**specs,N_agent = N)

        # Run the experiment on each algorithm
        for name,alg in algs.items():

            if executed[name][N]:
                continue
            
            print("Env:", env_name, "Alg:", name, "N agent:",N)

            result = {}
            
            def timeout_handler(signum,frame):
                print("Algorithm timeout")
                # Mark the rest of the executed table as True, so we don't ever visit this algorithm again
                for n in executed[name]:
                    if n>=N:
                        executed[name][n] = True
                # Save checkpoint data
                with open("./data/{}.pkl".format(env_name),'wb') as fp:
                    pkl.dump({'results':results, 'executed':executed},fp)
                raise TimeoutError("Timeout")
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                t0 = time()
                plan = alg(env)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["env"]=env

            results.append(result)

            # Mark the executed status as True
            executed[name][N] = True

            if t>TIMEOUT:
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                result['plan'] = plan
                print("On time for algorithm", name, "N agent = ", N)

            # Save checkpoint data
            with open("./data/{}.pkl".format(env_name),'wb') as fp:
                pkl.dump({'results':results, 'executed':executed},fp)
            
def get_checkpoint_state(env_name,algs,N_range,first_run = False):
    if first_run:
        results = []
        executed = {alg: {N :False for N in N_range} for alg in algs}
        # Save checkpoint state
        with open("./data/{}.pkl".format(env_name),'wb') as fp:
            pkl.dump({'results':results, 'executed':executed},fp)

    # Load checkpoint state
    with open("./data/{}.pkl".format(env_name),'rb') as fp:
        ckpt = pkl.load(fp)

    results = ckpt['results']
    executed = ckpt['executed']

    return results,executed


In [4]:
from panav.sequential import sequential_HybridSIPP, sequential_planning
bloating_r = 0.5
vmax = 1.0

TIMEOUT = 120

# Specify the Algorithms
algs = {  
        'SIPP':lambda HG: sequential_HybridSIPP(HG),
        "Seq_Tube": lambda HG: sequential_planning(Tube_Planning,HG.env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        }
# Load environment presets
env_names = [get_env_name(fn) for fn in listdir('./envs')]

env_info = {}

for name in env_names:
    with open(f'./envs/{name}.pkl','rb') as fp:
        env_info[name] = pkl.load(fp)

In [45]:
env_name = 'Room'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = [3,9,12,15]

In [46]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: Room Alg: SIPP N agent: 3
0
1
2
On time for algorithm SIPP N agent =  3
Env: Room Alg: Seq_Tube N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube N agent =  3
Building the environment...
Env: Room Alg: SIPP N agent: 9
0
1
2
3
4
5
6
7
8
MILP low-level encounters infeasibility.
On time for algorithm SIPP N agent =  9
Env: Room Alg: Seq_Tube N agent: 9
Planning for agent 0/9
Planning for agent 1/9
Planning for agent 2/9
Planning for agent 3/9
Planning for agent 4/9
Planning for agent 5/9
Planning for agent 6/9
Planning for agent 7/9
Planning for agent 8/9
On time for algorithm Seq_Tube N agent =  9
Building the environment...
Env: Room Alg: SIPP N agent: 12
0
1
2
3
4
5
6
7
8
MILP low-level encounters infeasibility.
9
10
11
On time for algorithm SIPP N agent =  12
Env: Room Alg: Seq_Tube N agent: 12
Planning for agent 0/12
Planning for agent 1/12
Planning for agent 2/12
Planning for agent 3/12
Pla

In [42]:
env_name = 'Warehouse'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = [9,12,15]

In [43]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: Warehouse Alg: SIPP N agent: 9
0
1
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
2
3
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters infeasibility.
MILP low-level encounters 

KeyboardInterrupt: 

In [38]:
env_name = 'MultiTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = [3,6,9,12,15]

In [39]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: MultiTunnel Alg: SIPP N agent: 3
0
1
2
On time for algorithm SIPP N agent =  3
Env: MultiTunnel Alg: Seq_Tube N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube N agent =  3
Building the environment...
Env: MultiTunnel Alg: SIPP N agent: 6
0
1
2
3
4
5
On time for algorithm SIPP N agent =  6
Env: MultiTunnel Alg: Seq_Tube N agent: 6
Planning for agent 0/6
Planning for agent 1/6
Planning for agent 2/6
Planning for agent 3/6
Planning for agent 4/6
Planning for agent 5/6
On time for algorithm Seq_Tube N agent =  6
Building the environment...
Env: MultiTunnel Alg: SIPP N agent: 9
0
1
2
3
4
5
6
7
8
On time for algorithm SIPP N agent =  9
Env: MultiTunnel Alg: Seq_Tube N agent: 9
Planning for agent 0/9
Planning for agent 1/9
Planning for agent 2/9
Planning for agent 3/9
Planning for agent 4/9
Planning for agent 5/9
Planning for agent 6/9
Planning for agent 7/9
Planning for agent 8/9
On time for algori

In [5]:
env_name = 'SingleTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = [3,6,9,12,15]

In [6]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)
# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: SingleTunnel Alg: SIPP N agent: 3
0
no need for soft planning at (0, 7) edge type soft (lb,ub) (0, inf)
no need for soft planning at (7, 6) edge type hard (lb,ub) (0, inf)
no need for soft planning at (6, 7) edge type hard (lb,ub) (0, inf)
no need for soft planning at (6, 3) edge type soft (lb,ub) (0, inf)
1
no need for soft planning at (1, 6) edge type soft (lb,ub) (29.965324407740383, inf)
no need for soft planning at (6, 7) edge type hard (lb,ub) (24.30282440774038, inf)
no need for soft planning at (7, 6) edge type hard (lb,ub) (29.965324407740383, inf)
no need for soft planning at (7, 4) edge type soft (lb,ub) (0, inf)
2
soft planning for (2, 6) edge type soft (lb,ub) (31.965324407740383, inf)
Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-15
[[-16.       -3.08125]
 [-18.       -0.25   ]]
[[-16.       -2.58977  -3.08125]
 [-18.       -2.14205  -0.25   ]]
[[-16.       -6.21958  -3.08125]
 [-18.        1.46457

/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)


[[ 16.      3.     -3.    -18.25 ]
 [ 18.      0.475  -0.475 -17.75 ]]
Planning for agent 1/3
[[-16.    17.75]
 [ 18.   -17.75]]
[[-16.     -3.      3.     17.75 ]
 [ 18.      0.475  -0.475 -17.75 ]]
Planning for agent 2/3
[[-16.    17.75]
 [-18.    17.75]]
[[-16.       -4.44789   3.       17.75   ]
 [-18.        0.475     0.475    17.75   ]]
On time for algorithm Seq_Tube N agent =  3
Building the environment...
Env: SingleTunnel Alg: SIPP N agent: 6
0
no need for soft planning at (0, 13) edge type soft (lb,ub) (0, inf)
no need for soft planning at (13, 12) edge type hard (lb,ub) (0, inf)
no need for soft planning at (12, 13) edge type hard (lb,ub) (0, inf)
no need for soft planning at (12, 6) edge type soft (lb,ub) (0, inf)
1
no need for soft planning at (1, 13) edge type soft (lb,ub) (0, 20.30282440774038)
no need for soft planning at (1, 13) edge type soft (lb,ub) (24.30282440774038, inf)
no need for soft planning at (13, 12) edge type hard (lb,ub) (0, 25.965324407740383)
no need f